## AI Proxy

This project is designed to make it easy to send the same prompt to multiple LLMs which is useful for testing and comparison.

### API Access Required

You must have access to the services (Currently OpenAI, Google, and Perplexity) in order to use them in this script.

- Google GenerativeAI Python Library - `pip install -q google.generativeai`
- Tools to grab API key from Environment variable: `pip install python-dotenv`

### Working with API keys

You will need to set the Gemini API key as a system variable named: `GOOGLE_API_KEY`.  

- [Setting an Environment Variable on Mac/Linux](https://phoenixnap.com/kb/set-environment-variable-mac)
- [Setting an Environment Variable on Windows](https://phoenixnap.com/kb/windows-set-environment-variable)

#### Import Google Generative GenerativeAI library and set API Key

## Tools to Get Environment Variables from OS

PIP Install:

`pip install python-dotenv`

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

## Setup Google GenAI

PIP Install: 

`pip install -q google.generativeai`

In [ ]:
import google.generativeai as googleai

_ = load_dotenv(find_dotenv()) # read local .env file
apiKey = os.getenv('GOOGLE_API_KEY')

googleai.configure(api_key=apiKey,
               transport="rest",
    )

## Explore the Available Models

Learn which models are currently available


In [ ]:
for m in googleai.list_models():
    print(f"name: {m.name}")
    print(f"description: {m.description}")
    print(f"generation methods:{m.supported_generation_methods}\n")

### Filter models to ensure model we want is supported
- `generateContent` is the value we are looking for

In [ ]:
for m in googleai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

### Google AI Helper Function

- The `@retry` decorator helps you to retry the API call if it fails.

In [ ]:
from google.api_core import retry
@retry.Retry()
def generate_text_google(prompt, model):
    model = googleai.GenerativeModel(model)
    response = model.generate_content(prompt)
    return response.text

### Test **Google AI Helper** function

In [ ]:
generate_text = generate_text_google("Thursday evenings are perfect for", "gemini-pro")
print(generate_text)

## Setup Open AI APIs

```
OpenAI's APIs offer developers the ability to integrate advanced artificial intelligence capabilities into their applications, enabling a wide range of tasks from text generation to complex problem-solving.
```
Documentation: [https://beta.openai.com/docs/](https://beta.openai.com/docs/)

### Obtaining API Keys:
- **OpenAI Platform**: [https://platform.openai.com/](https://platform.openai.com/)
  - After signing up or logging in, navigate to the API section to manage and obtain your API keys.
- You will need to set the OpenAI API key as a system variable named: `OPENAI_API_KEY`.  

Note: do NOT check your API key into a public Github repo, or it will get revoked 
  
  


In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

### Open AI Helper Function

PIP Dependencies:

`pip install --upgrade openai`

In [ ]:
from openai import OpenAI
client = OpenAI()

def generate_text_openai(pre, prompt, model):
    completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": pre},
        {"role": "user", "content": prompt}
    ]
    )

    return completion.choices[0].message.content

## Test **Open AI Helper** Function

In [ ]:
print(generate_text_openai("You are a pirate", "Thursday evenings are perfect for", "gpt-3.5-turbo"))


## Setup Perplexity API

You will need a key set to `PERPLEXITY_API_KEY`

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

YOUR_API_KEY = os.getenv('PERPLEXITY_API_KEY')

## Perplexity Helper function

No PIP dependency

In [ ]:
from openai import OpenAI

perplexityClient = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")

def generate_text_perplexity(system, user, model):
    response = perplexityClient.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system},
        {"role": "user", "content": user}
    ]
    )

    content = response.choices[0].message.content
    return content


## Test **Perplexity Helper** Function

In [ ]:
print(generate_text_perplexity("you are a pirate", "say hello and return the message in uppercase", "mistral-7b-instruct"))

## Proxy to Previously Defined Functions

In [ ]:
def log_prompt(system, user, format):
    print("**** PROMPT ****")
    print(f"** system **:\n{system}\n\n** user **\n{user}\n\n** format **\n{format}")
    print("**** END PROMPT ****\n")

def log_request(model):
    print(f"**** BEGIN: {model} *****")

def log_response(response, model = ""):
    print(f"*** RESPONSE:  {model} ****\n{response}\n")

1. Define a function for each model you want to test
2. Create a constant to reference that model
3. Add both to the dictionary

In [ ]:
# Function for each model to test
def action_openai_35turbo(system, user, format):
    response = generate_text_openai(system, user + format, "gpt-3.5-turbo")
    log_response(response, "gpt-3.5-turbo")

def action_openai_gpt4(system, user, format):
    response = generate_text_openai(system, user + format, "gpt-4")
    log_response(response, "gpt-4")

def action_openai_gpt4_preview(system, user, format):
    response = generate_text_openai(system, user + format, "gpt-4-0125-preview")
    log_response(response, "gpt-4-0125-preview")

def action_gemini_pro(system, user, format,):
    response = generate_text_google(system + user + format, "gemini-pro")
    log_response(response, "gemini-pro")

def action_mistral_7b(system, user, format):
    response = generate_text_perplexity(system, user + format, "mistral-7b-instruct")
    log_response(response, "mistral-7b-instruct")

def action_mixtral_8x7b(system, user, format):
    response = generate_text_perplexity(system, user + format, "mixtral-8x7b-instruct")
    log_response(response, "mixtral-8x7b-instruct")

# Constants for the models
OPEN_AI_35TURBO = "gpt-3.5-turbo"
OPEN_AI_GPT4 = "gpt-4"
OPEN_AI_GPT4PREVIEW = "gpt-4-0125-preview"
GEMINI_PRO = "gemini-pro"
MISTRAL_7B = "mistral-7b-instruct"
MIXTRAL_8X7B = "mixtral-8x7b-instruct"

# Dictionary mapping models to their respective functions
action_dict = {
    OPEN_AI_35TURBO: action_openai_35turbo,
    OPEN_AI_GPT4: action_openai_gpt4,
    OPEN_AI_GPT4PREVIEW: action_openai_gpt4_preview,
    GEMINI_PRO: action_gemini_pro,
    MISTRAL_7B: action_mistral_7b,
    MIXTRAL_8X7B: action_mixtral_8x7b
}

        

In [ ]:
## Log the prompt and call the respective model functions
def generate_text(models, system, user, format):
    log_prompt(system, user, format)
    for model in models:
        action = action_dict.get(model)
        if action:
            log_request(model=model)
            action(system=system, user=user, format=format)
        else:
            print("No action defined for model: ", model)

## Test AI Proxy



### Define Prompt components

### `system` - defines context of prompt

In [ ]:
system = """
You are a art expert speaking at a symposium in New York, NY.
"""


### `user` - defines the question they are asking

In [ ]:
user = """
tell me about the most interesting public art piece in town
"""

### `format` - defines post processing and what the format of the output should be

In [ ]:
format = """
be sure to mention the location of the art, and the artist name.  Use proper markdown format for the output.
"""



## Call list of models using common prompt components

This will send same 'system', 'user' and 'format' prompt components

Each model in the list will be called, remove LLMs you don't want to be called.

In [ ]:
# Possible Models to use
# OPEN_AI_35TURBO,
# OPEN_AI_GPT4,
# OPEN_AI_GPT4PREVIEW,
# GEMINI_PRO,
# MISTRAL_7B,
# MIXTRAL_8X7B
models = [
    OPEN_AI_35TURBO,
    GEMINI_PRO
]

generate_text(models, system, user, format)